# Tutorial 3: Volatility Modeling with GARCH

This tutorial demonstrates how to model and forecast financial volatility using GARCH models.

**Models Covered:**
- GARCH - Generalized Autoregressive Conditional Heteroskedasticity
- EGARCH - Exponential GARCH (asymmetric volatility)
- GJR-GARCH - Threshold GARCH

**Dataset:** Synthetic daily financial returns with volatility clustering

**Applications:**
- Risk management (VaR, CVaR)
- Portfolio optimization
- Option pricing
- Trading strategy development

## Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Import KRL Model Zoo volatility models
from krl_models.volatility import GARCHModel, EGARCHModel, GJRGARCHModel
from krl_core import ModelMeta

# Set display options
pd.set_option('display.max_columns', None)
plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## Load Financial Returns Data

In [ ]:
# Load returns data
df = pd.read_csv('../data/financial_returns.csv')
df['date'] = pd.to_datetime(df['date'])

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['date'].min()} to {df['date'].max()}")
print(f"\nReturn statistics:")
print(df['returns'].describe())
df.head()

In [ ]:
# Visualize returns
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# Returns over time
axes[0].plot(df['date'], df['returns'], linewidth=0.8, alpha=0.7)
axes[0].set_title('Daily Returns', fontsize=14, fontweight='bold')
axes[0].set_ylabel('Returns (%)')
axes[0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0].grid(True, alpha=0.3)

# Returns distribution
axes[1].hist(df['returns'], bins=50, edgecolor='black', alpha=0.7)
axes[1].set_title('Returns Distribution', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Returns (%)')
axes[1].set_ylabel('Frequency')
axes[1].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Check for volatility clustering
print("\nVolatility Clustering Test:")
print("Large returns followed by large returns (in absolute value) = volatility clustering")

## Model 1: GARCH(1,1) - Standard Volatility Model

GARCH (Generalized Autoregressive Conditional Heteroskedasticity) models time-varying volatility.

**GARCH(1,1) equation:**
- variance_t = omega + alpha * epsilon_(t-1)^2 + beta * variance_(t-1)

**Interpretation:**
- omega: Base volatility level
- alpha: Reaction to shocks (ARCH effect)
- beta: Persistence of volatility (GARCH effect)

In [ ]:
# Split data
train_size = int(len(df) * 0.8)
train_data = df[:train_size].copy()
test_data = df[train_size:].copy()

print(f"Training observations: {len(train_data)}")
print(f"Test observations: {len(test_data)}")

In [ ]:
# Configure GARCH(1,1) model
garch_params = {
    'returns_col': 'returns',
    'p': 1,  # GARCH order
    'q': 1   # ARCH order
}

meta = ModelMeta(
    name="Returns_GARCH",
    version="1.0",
    author="Tutorial",
    description="GARCH(1,1) model for volatility forecasting"
)

# Fit GARCH model
garch = GARCHModel(train_data, garch_params, meta)
result = garch.fit()

print("\nGARCH(1,1) Model fitted successfully!")
print(f"\nModel Parameters:")
print(f"  omega: {result.payload['params']['omega']:.6f}")
print(f"  alpha[1]: {result.payload['params']['alpha[1]']:.4f}")
print(f"  beta[1]: {result.payload['params']['beta[1]']:.4f}")
print(f"\nPersistence (alpha + beta): {result.payload['params']['alpha[1]'] + result.payload['params']['beta[1]']:.4f}")
print(f"Log-Likelihood: {result.payload['log_likelihood']:.2f}")
print(f"AIC: {result.payload['aic']:.2f}")

In [ ]:
# Extract fitted conditional volatility
fitted_volatility = result.payload['conditional_volatility']

# Visualize fitted volatility
plt.figure(figsize=(14, 6))

plt.subplot(2, 1, 1)
plt.plot(train_data['date'], train_data['returns'], linewidth=0.8, alpha=0.7, label='Returns')
plt.title('Returns', fontsize=14, fontweight='bold')
plt.ylabel('Returns (%)')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(2, 1, 2)
plt.plot(train_data['date'], fitted_volatility, linewidth=1.5, color='red', label='Conditional Volatility')
plt.title('GARCH(1,1) Conditional Volatility', fontsize=14, fontweight='bold')
plt.ylabel('Volatility')
plt.xlabel('Date')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Forecast volatility
forecast = garch.predict(train_data, steps=len(test_data))
forecast_volatility = forecast.payload['forecast_volatility']

print(f"Volatility forecast shape: {len(forecast_volatility)} periods")
print(f"Mean forecast volatility: {np.mean(forecast_volatility):.4f}")
print(f"Max forecast volatility: {np.max(forecast_volatility):.4f}")

In [ ]:
# Visualize volatility forecast
plt.figure(figsize=(14, 6))

# Historical volatility
plt.plot(train_data['date'].iloc[-100:], fitted_volatility[-100:], 
         label='Historical Volatility', color='blue', linewidth=2)

# Forecast volatility
plt.plot(test_data['date'], forecast_volatility, 
         label='Forecast Volatility', color='red', linewidth=2, linestyle='--')

# Actual absolute returns (proxy for realized volatility)
plt.plot(test_data['date'], np.abs(test_data['returns']), 
         label='Absolute Returns', color='green', alpha=0.5, linewidth=1)

plt.title('GARCH(1,1) Volatility Forecast', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Model 2: EGARCH - Asymmetric Volatility

EGARCH (Exponential GARCH) captures the leverage effect: negative shocks increase volatility more than positive shocks of the same magnitude.

**Why EGARCH?**
- Captures asymmetry (bad news has larger impact)
- No parameter constraints (always positive variance)
- Better for equity returns

In [ ]:
# Configure EGARCH model
egarch_params = {
    'returns_col': 'returns',
    'p': 1,
    'q': 1
}

meta_egarch = ModelMeta(
    name="Returns_EGARCH",
    version="1.0",
    author="Tutorial",
    description="EGARCH(1,1) for asymmetric volatility"
)

# Fit EGARCH
egarch = EGARCHModel(train_data, egarch_params, meta_egarch)
egarch_result = egarch.fit()

print("EGARCH(1,1) Model fitted!")
print(f"\nAsymmetry parameter (gamma): {egarch_result.payload['params'].get('gamma[1]', 'N/A')}")
print(f"  gamma < 0: Leverage effect exists")
print(f"  gamma = 0: No asymmetry")
print(f"  gamma > 0: Inverse leverage effect")
print(f"\nAIC: {egarch_result.payload['aic']:.2f}")

In [ ]:
# Forecast with EGARCH
egarch_forecast = egarch.predict(train_data, steps=len(test_data))
egarch_volatility = egarch_forecast.payload['forecast_volatility']

# Compare GARCH vs EGARCH
plt.figure(figsize=(14, 6))

plt.plot(test_data['date'], forecast_volatility, 
         label='GARCH Forecast', color='red', linewidth=2, linestyle='--')
plt.plot(test_data['date'], egarch_volatility, 
         label='EGARCH Forecast', color='purple', linewidth=2, linestyle=':')
plt.plot(test_data['date'], np.abs(test_data['returns']), 
         label='Absolute Returns', color='green', alpha=0.5, linewidth=1)

plt.title('GARCH vs EGARCH Volatility Forecasts', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Model 3: GJR-GARCH - Threshold GARCH

GJR-GARCH adds a threshold term to capture asymmetry in a different way than EGARCH.

**Equation:**
- variance_t = omega + (alpha + gamma * I_(t-1)) * epsilon_(t-1)^2 + beta * variance_(t-1)
- I_(t-1) = 1 if epsilon_(t-1) < 0, else 0

**Interpretation:**
- gamma > 0: Negative shocks increase volatility more

In [ ]:
# Configure GJR-GARCH
gjr_params = {
    'returns_col': 'returns',
    'p': 1,
    'o': 1,  # Threshold order
    'q': 1
}

meta_gjr = ModelMeta(
    name="Returns_GJRGARCH",
    version="1.0",
    author="Tutorial",
    description="GJR-GARCH(1,1,1) for threshold effects"
)

# Fit GJR-GARCH
gjr = GJRGARCHModel(train_data, gjr_params, meta_gjr)
gjr_result = gjr.fit()

print("GJR-GARCH(1,1,1) Model fitted!")
print(f"\nThreshold parameter (gamma): {gjr_result.payload['params'].get('gamma[1]', 'N/A')}")
print(f"AIC: {gjr_result.payload['aic']:.2f}")

In [ ]:
# Forecast with GJR-GARCH
gjr_forecast = gjr.predict(train_data, steps=len(test_data))
gjr_volatility = gjr_forecast.payload['forecast_volatility']

# Three-way comparison
plt.figure(figsize=(14, 6))

plt.plot(test_data['date'], forecast_volatility, 
         label='GARCH', color='red', linewidth=2, linestyle='--', alpha=0.7)
plt.plot(test_data['date'], egarch_volatility, 
         label='EGARCH', color='purple', linewidth=2, linestyle=':', alpha=0.7)
plt.plot(test_data['date'], gjr_volatility, 
         label='GJR-GARCH', color='orange', linewidth=2, linestyle='-.', alpha=0.7)
plt.plot(test_data['date'], np.abs(test_data['returns']), 
         label='Absolute Returns', color='green', alpha=0.5, linewidth=1)

plt.title('Volatility Model Comparison', fontsize=16, fontweight='bold')
plt.xlabel('Date')
plt.ylabel('Volatility')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Model Comparison & Evaluation

In [ ]:
# Compare model fit
comparison = pd.DataFrame({
    'Model': ['GARCH(1,1)', 'EGARCH(1,1)', 'GJR-GARCH(1,1,1)'],
    'Log-Likelihood': [
        result.payload['log_likelihood'],
        egarch_result.payload['log_likelihood'],
        gjr_result.payload['log_likelihood']
    ],
    'AIC': [
        result.payload['aic'],
        egarch_result.payload['aic'],
        gjr_result.payload['aic']
    ],
    'BIC': [
        result.payload.get('bic', np.nan),
        egarch_result.payload.get('bic', np.nan),
        gjr_result.payload.get('bic', np.nan)
    ]
})

print("Model Comparison (In-Sample Fit):")
print(comparison.to_string(index=False))
print("\nLower AIC/BIC = Better fit")

In [ ]:
# Calculate forecast accuracy (using squared returns as volatility proxy)
squared_returns = test_data['returns'].values ** 2

mse_garch = np.mean((squared_returns - forecast_volatility**2) ** 2)
mse_egarch = np.mean((squared_returns - egarch_volatility**2) ** 2)
mse_gjr = np.mean((squared_returns - gjr_volatility**2) ** 2)

print("\nForecast Accuracy (MSE on squared returns):")
print(f"  GARCH:     {mse_garch:.6f}")
print(f"  EGARCH:    {mse_egarch:.6f}")
print(f"  GJR-GARCH: {mse_gjr:.6f}")
print(f"\nBest forecast model: {['GARCH', 'EGARCH', 'GJR-GARCH'][np.argmin([mse_garch, mse_egarch, mse_gjr])]}")

## Practical Applications

### 1. Value at Risk (VaR) Calculation

In [ ]:
# Calculate 1-day 95% VaR using GARCH forecast
confidence_level = 0.95
z_score = 1.645  # 95% confidence (one-tailed)

# VaR for next period
next_volatility = forecast_volatility[0]
VaR_95 = z_score * next_volatility

print(f"1-Day 95% Value at Risk:")
print(f"  Forecast volatility: {next_volatility:.4f}")
print(f"  VaR (95%): {VaR_95:.4f}")
print(f"\nInterpretation: With 95% confidence, losses will not exceed {VaR_95:.2f}% in the next day")

### 2. Expected Shortfall (CVaR)

In [ ]:
# Expected Shortfall (average loss beyond VaR)
# For normal distribution: ES = sigma * phi(z) / (1-alpha)
from scipy.stats import norm

phi_z = norm.pdf(z_score)
ES_95 = next_volatility * phi_z / (1 - confidence_level)

print(f"1-Day 95% Expected Shortfall (CVaR):")
print(f"  ES (95%): {ES_95:.4f}")
print(f"\nInterpretation: Average loss in worst 5% scenarios is {ES_95:.2f}%")

## Key Takeaways

1. **GARCH Models** capture volatility clustering - periods of high volatility followed by high volatility

2. **Model Selection:**
   - **GARCH**: Simple, interpretable, good baseline
   - **EGARCH**: Asymmetric effects, no parameter constraints
   - **GJR-GARCH**: Threshold effects, easier interpretation than EGARCH

3. **Leverage Effect:** Negative returns typically increase volatility more than positive returns
   - Check gamma parameter in EGARCH/GJR-GARCH

4. **Persistence:** alpha + beta close to 1 means volatility shocks persist
   - High persistence = slow mean reversion
   - Low persistence = quick return to baseline

5. **Applications:**
   - Risk management (VaR, CVaR)
   - Portfolio optimization
   - Option pricing inputs
   - Trading strategies (volatility targeting)

## Next Steps

- Test higher-order GARCH models (p>1, q>1)
- Compare with realized volatility (if available)
- Implement backtesting for VaR models
- Explore multivariate GARCH (DCC-GARCH)
- Use volatility forecasts for portfolio rebalancing